https://beautybliss.co.nz/

In [144]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import time

# url ของเว็บที่ต้องการดูข้อมูล
url = "https://beautybliss.co.nz/shop/australis-fresh-flawless-pressed-powder/" 

# Set up EdgeOptions
edge_options = Options()
edge_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.54")
edge_options.add_argument("window-size=1920x1080")
edge_options.add_argument("disable-extensions")
edge_options.add_argument("disable-site-isolation-trials")

driver = webdriver.Edge(options=edge_options)
driver.get(url)

In [145]:
# review_blocks = (driver.find_element(By.CLASS_NAME, 'card-container')).text
# review_blocks

In [146]:
load_more_buttons = (driver.find_element(By.ID, "ReviewsLoadMore")).text
load_more_buttons

'Load More'

In [147]:
while True:
    try:
        # Scroll ลงล่างเพื่อให้ปุ่มแสดง
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

        # หา Load More ด้วย ID
        load_more_button = driver.find_element(By.ID, "ReviewsLoadMore")

        if load_more_button.is_displayed() and load_more_button.is_enabled():
            print("Clicking Load More...")
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(2)
        else:
            print("No more reviews to load.")
            break

    except NoSuchElementException:
        print("No Load More button found.")
        break

    except Exception as e:
        print("Unexpected error:", str(e))
        break

Clicking Load More...
Clicking Load More...
Clicking Load More...
No Load More button found.


In [148]:
# content = driver.find_elements(By.TAG_NAME, 'small')
# dates = [date.text.strip() for date in content if date.text.strip() != ""]
# dates

In [149]:
# content = driver.find_elements(By.TAG_NAME, 'strong')
# headers = [header.text.strip() for header in content if header.text.strip() != ""]
# headers

In [150]:
# content = driver.find_elements(By.CLASS_NAME, 'review-text')
# reviews = [review.text for review in content]
# reviews

In [151]:
# print(f"dates  : {len(dates)} \nheaders: {len(headers)} \nreviews: {len(reviews)}")

In [152]:
# import pandas as pd

# df = pd.DataFrame({
#     'date': dates,
#     'title': headers,
#     'content': reviews
# })

# df.head()

In [153]:
review_blocks = driver.find_elements(By.XPATH, '//div[@class="row"]/div[contains(@class, "col-3") or contains(@class, "col-9")]')

for i, block in enumerate(review_blocks):
    print(f"---- Block {i+1} ----")
    print(block.text.strip())

---- Block 1 ----

---- Block 2 ----

---- Block 3 ----

---- Block 4 ----

---- Block 5 ----

---- Block 6 ----

---- Block 7 ----
S

Stacey
7 months ago
---- Block 8 ----
Shade: Natural
Fave pressed powder
I have been using this pressed powder since i was 18. I am not 36 and still using it. Nothing else compares.
---- Block 9 ----
Katrina
8 months ago
---- Block 10 ----
Shade: Light Beige
Love this
Love this product in this brand
---- Block 11 ----
Anna
9 months ago
---- Block 12 ----
Shade: Light Beige
Ride or die
The only product I hit pan on regularly and repurchase over and over again. Amazing for the under eye and contouring. Amazing formula. Full coverage. Better then Mac. This will forever be a staple and I pray they never stop making it. Love love love.
---- Block 13 ----
Natalie
9 months ago
---- Block 14 ----
Shade: Light Beige
Love!
I love this it’s so good!
---- Block 15 ----
A

Ash
1 year ago
---- Block 16 ----
Shade: Natural
Staple
My everyday staple powder 🥰
---- Block

In [154]:
parsed_reviews = []

# 2 บล็อค = 1 รีวิว
for i in range(0, len(review_blocks), 2):
    if i + 1 < len(review_blocks):
        block1 = review_blocks[i].text.strip().split("\n")
        block2 = review_blocks[i + 1].text.strip().split("\n")

        # หา date จาก block1
        date = next((line for line in block1 if "ago" in line), None)

        # title และ review มาจาก block2
        title = block2[1] if len(block2) > 1 else None
        review = "\n".join(block2[2:]) if len(block2) > 2 else None

        parsed_reviews.append({
            "date": date,
            "title": title,
            "review": review
        })

df = pd.DataFrame(parsed_reviews)

df = df[df['review'].notnull() & df['review'].str.strip().ne("")].reset_index(drop=True)

df.head()

,date,title,review
0,7 months ago,Fave pressed powder,I have been using this pressed powder since i ...
1,8 months ago,Love this,Love this product in this brand
2,9 months ago,Ride or die,The only product I hit pan on regularly and re...
3,9 months ago,Love!,I love this it’s so good!
4,1 year ago,Staple,My everyday staple powder 🥰


In [155]:
len(df)

62

In [156]:
def convert_to_date(text):
    today = datetime.today()
    try:
        if "year" in text:
            num = int(text.split()[0])
            return (today - relativedelta(years=num)).strftime('%Y-%m-%d')
        elif "month" in text:
            num = int(text.split()[0])
            return (today - relativedelta(months=num)).strftime('%Y-%m-%d')
        elif "week" in text:
            num = int(text.split()[0])
            return (today - relativedelta(weeks=num)).strftime('%Y-%m-%d')
        elif "day" in text:
            num = int(text.split()[0])
            return (today - relativedelta(days=num)).strftime('%Y-%m-%d')
        else:
            return text  # อาจเป็นวันที่ตรง ๆ
    except:
        return text

df['date'] = df['date'].apply(convert_to_date)
df.head()

,date,title,review
0,2024-10-20,Fave pressed powder,I have been using this pressed powder since i ...
1,2024-09-20,Love this,Love this product in this brand
2,2024-08-20,Ride or die,The only product I hit pan on regularly and re...
3,2024-08-20,Love!,I love this it’s so good!
4,2024-05-20,Staple,My everyday staple powder 🥰


In [157]:
df['source'] = 'beautybliss.co.nz'
df['product'] = 'Australis Fresh & Flawless Pressed Powder'

In [158]:
df.head()

,date,title,review,source,product
0,2024-10-20,Fave pressed powder,I have been using this pressed powder since i ...,beautybliss.co.nz,Australis Fresh & Flawless Pressed Powder
1,2024-09-20,Love this,Love this product in this brand,beautybliss.co.nz,Australis Fresh & Flawless Pressed Powder
2,2024-08-20,Ride or die,The only product I hit pan on regularly and re...,beautybliss.co.nz,Australis Fresh & Flawless Pressed Powder
3,2024-08-20,Love!,I love this it’s so good!,beautybliss.co.nz,Australis Fresh & Flawless Pressed Powder
4,2024-05-20,Staple,My everyday staple powder 🥰,beautybliss.co.nz,Australis Fresh & Flawless Pressed Powder


In [143]:
df.to_excel(f"1_{df['product'].iloc[0]}_2.xlsx", index=False)